In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import count,col,when, collect_list,concat_ws,lit,monotonically_increasing_id,first
from pyspark.sql.dataframe import *
from pyspark.sql.types import TimestampType, StringType
from pyspark.sql import SparkSession, functions as F

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os

# Credentials MinIO
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["AWS_REGION"] = "eu-west-1"

conf = (
    SparkConf()
    .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .set("spark.sql.catalog.warehouse", "org.apache.iceberg.spark.SparkCatalog")
    .set("spark.sql.catalog.warehouse.type", "rest")
    .set("spark.sql.catalog.warehouse.uri", "http://rest:8181/")  # nom du service REST
    .set("spark.sql.catalog.warehouse.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .set("spark.sql.catalog.warehouse.s3.endpoint", "http://minio:9000")  # nom du service MinIO
    .set("spark.sql.catalog.warehouse.warehouse", "s3://iceberg-lakehouse/wh")  # bucket déjà créé
    .set("spark.hadoop.fs.s3a.path.style.access", "true")  # nécessaire pour MinIO
   
)

spark = SparkSession.builder.config(conf=conf).appName("IcebergNotebook").getOrCreate()
spark.sparkContext.setLogLevel("DEBUG")

In [ ]:
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType
schema = StructType([
  StructField("vendor_id", LongType(), True),
  StructField("trip_id", LongType(), True),
  StructField("trip_distance", FloatType(), True),
  StructField("fare_amount", DoubleType(), True),
  StructField("store_and_fwd_flag", StringType(), True)
])

df = spark.createDataFrame([], schema)
df.writeTo("warehouse.nyc.taxis").createOrReplace()

In [ ]:
schema = spark.table("warehouse.nyc.taxis").schema
data = [
    (1, 1000371, 1.8, 15.32, "N"),
    (2, 1000372, 2.5, 22.15, "N"),
    (2, 1000373, 0.9, 9.01, "N"),
    (1, 1000374, 8.4, 42.13, "Y")
  ]
df = spark.createDataFrame(data, schema)
df.writeTo("warehouse.nyc.taxis").append()

In [ ]:
df = spark.table("warehouse.nyc.taxis").show()

In [ ]:
spark.sql("SHOW TBLPROPERTIES warehouse.nyc.taxis;").show()
